# Modélisation du système bactérie/anticorps

Nadir Ait Kheddache, Idriss Alaoui Souliman, Fabien Benthami, Alma Draeger, Alexandre Janin

In [1]:
#Imports
import numpy as np
import ipywidgets as widgets

In [2]:
#Variables globales
#probabilité d'avoir chaque type de cellule (0: vide, 1: bactérie, 2: globule blanc, 3: nutriments)
#la somme des probabilités doit être égale à 1
proba = [0.5, 0.1, 0.1, 0.3]

#Symbole qui représente chaque valeur de cellule
cara = ['-', 'B', 'G', 'n']

In [3]:
#Paramètres
infiniCheckbox = widgets.Checkbox(value = True, description = "Infini", disabled = False)
display(infiniCheckbox)

Checkbox(value=True, description='Infini')

In [4]:
def matrice(l, h):
    """ int^2 -> int[][]
    largeur > 0, hauteur > 0
    Renvoie une matrice de 0 de l colonnes et h lignes"""
    
    return [[aleatoire() for x in range(l)] for y in range(h)]

In [5]:
def aleatoire():
    """float -> int
    Renvoie le type de cellule (0, 1, 2, 3) selon le nombre aléatoire tiré"""
    
    alea = np.random.rand()
    
    for i in range(len(proba)):
        p = proba[i]
        if alea <= p:
            return i
        else:
            alea -= p
            
    return 0

In [6]:
def printmatrice(M):
    """int[][] -> NoneType
    Affiche la matrice M"""
    for ligne in M:
        s = ""
        for cell in ligne:
            s += cara[cell] + ' '
        print(s + "\n")

In [7]:
def voisinage(M, x, y):
    """int[][] * int * int * bool-> int[]
    Renvoie la liste des voisins de la case (x, y).
    Si infini est False, on ne prend pas les voisins hors de la matrice"""
    v = [0, 0, 0, 0]
    
    ymax = len(M)
    xmax = len(M[0])
    
    infini = infiniCheckbox.value
    
    for i in [x - 1, x, x + 1]:
        if (infini or (0 <= i and i < xmax)):
            for j in [y - 1, y, y + 1]:
                if (infini or (0 <= j and j < ymax)):
                        if (i, j) != (x, y): #Si i, j n'est pas la case centrale
                            v[M[j][i]] += 1
    return v

In [10]:
mat = matrice(9,9)
printmatrice(mat)

n - - n - G - n - 

G B - - B G n B n 

G - - G n - - - - 

n - n - - - - n G 

- G - - G B - n - 

- n - - B - n - - 

- - - n G - - - B 

G B n - - - n n n 

- G n - - B n - n 



In [9]:
print(voisinage(mat, 0, 5))

[2, 1, 3, 2]
